In [1]:
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
from tqdm import tqdm
import pandas as pd
import itertools
import lightgbm as lgb
from joblib import Parallel, delayed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
import pandas as pd
train_df = pd.read_csv('train.csv')
submit = pd.read_csv('sample_submission.csv')
brand_keyword_data = pd.read_csv('brand_keyword_cnt.csv')

In [ ]:
###########################################################################
###########################################################################
###########################################################################
###############################피처엔지니어링###############################
###########################################################################
###########################################################################
###########################################################################

In [8]:
###########################################################################
###########################################################################
###########################################################################
###############################타겟값 만들기################################
###########################################################################
###########################################################################
###########################################################################

def Make_Target(product_id, product_data):
    product_features = []
    for i in range(len(product_data) - 28):
        features = {
            'ID': product_id,
        }
        future_21_day_sales = product_data['sales'].iloc[i+28:i+49].values
        for j, target in enumerate(future_21_day_sales, start=1):
            features[f'target_{j}'] = target
        product_features.append(features)
    return product_features
def create_features_for_single_day(product_data):
    window_data = product_data.iloc[-28:]
    features = {
    }
    return features
def prepare_train_test_data_per_id(train_data, product_id):
    filtered_data = train_data[train_data['ID'] == product_id]
    melted_sales_data = filtered_data.melt(id_vars=['ID', '제품', '대분류', '중분류', '소분류', '브랜드'], 
                                           var_name='date', value_name='sales')
    melted_sales_data['date'] = pd.to_datetime(melted_sales_data['date'])
    product_features = Make_Target(product_id, melted_sales_data[melted_sales_data['date'] < '2023-04-04'])
    train_df_id = pd.DataFrame(product_features)
    train_df_id = train_df_id.merge(filtered_data[['ID', '제품', '대분류', '중분류', '소분류', '브랜드']].drop_duplicates(), on='ID', how='left')
    window_data = melted_sales_data.iloc[-28:]
    single_day_features = create_features_for_single_day(window_data)
    single_day_features['ID'] = product_id
    single_day_features['date'] = '2023-04-04'
    test_df_id = pd.DataFrame([single_day_features])
    test_df_id = test_df_id.merge(filtered_data[['ID', '제품', '대분류', '중분류', '소분류', '브랜드']].drop_duplicates(), on='ID', how='left')
    return train_df_id, test_df_id
def prepare_train_test_data(train_data):
    all_train_dfs = []
    all_test_dfs = []
    for product_id in tqdm(train_data['ID'].unique(), desc='Processing unique product IDs'):
        train_df_id, test_df_id = prepare_train_test_data_per_id(train_data, product_id)
        all_train_dfs.append(train_df_id)
        all_test_dfs.append(test_df_id)
    final_train_df = pd.concat(all_train_dfs, ignore_index=True)
    final_test_df = pd.concat(all_test_dfs, ignore_index=True)
    return final_train_df, final_test_df
final_train_df, final_test_df = prepare_train_test_data(train_df)

Processing unique product IDs:  10%|▉         | 1560/15890 [00:52<07:57, 30.04it/s]

In [ ]:
import xgboost as xgb

def train_predict_single_id(train_df_id, test_df_id):
    predictions_id = []
    for i in range(1, 22):
        target_column = f'target_{i}'

        other_target_columns = [f'target_{j}' for j in range(1, 22) if j != i]
        train_features = train_df_id.drop([target_column, 'date'] + other_target_columns, axis=1)
        train_target = train_df_id[target_column]

        test_features = test_df_id[train_features.columns]

        dtrain = xgb.DMatrix(train_features, label=train_target)
        params = {
            'eval_metric': 'rmse',
        }
        model = xgb.train(params, dtrain, 100)
        dtest = xgb.DMatrix(test_features)
        predictions_id.append(model.predict(dtest)[0])

    return predictions_id

predictions_df = pd.DataFrame(columns=['ID'] + [f'2023-04-{i+5:02d}' for i in range(21)])

for unique_id in tqdm(train_df['ID'].unique(), desc='Predicting for each ID'):
    train_df_id = final_train_df[final_train_df['ID'] == unique_id]
    test_df_id = final_test_df[final_test_df['ID'] == unique_id]
    predictions_id = train_predict_single_id(train_df_id, test_df_id)
    predictions_df.loc[len(predictions_df)] = [unique_id] + predictions_id

final_submission = submit[['ID']].merge(predictions_df, on='ID', how='left')

Predicting for each ID:   0%|          | 0/15890 [00:00<?, ?it/s]

In [ ]:
final_submission = submit[['ID']].merge(predictions_df, on='ID', how='left')
final_submission

In [ ]:
new_columns = [f'2023-04-{i:02d}' for i in range(5, 26)]
final_submission.columns = ['ID'] + new_columns
final_submission

In [112]:
final_submission.to_csv('XGBoost.csv', index=False)